In [20]:
import os
import pandas as pd
import requests
from io import BytesIO
from google.colab import files

Set directory and create if not present

In [19]:
DIRECTORY = '/content'
os.makedirs(DIRECTORY, exist_ok=True)

Truncate extreme values for TMB, Age, and NLR to avoid SettingWithCopyWarning.



In [3]:
def truncate_values(df):
  df['TMB'] = df['TMB'].clip(upper=50)
  df['Age'] = df['Age'].clip(upper=85)
  df['NLR'] = df['NLR'].clip(upper=25)

  return df

Select specified columns and truncate extreme values.

In [4]:
def select_and_truncate_columns(selected_columns, df):
  df_selected = df.loc[:, selected_columns].copy()
  df_selected = truncate_values(df_selected)

  return df_selected

Return selected columns based on response presence

In [5]:
def get_feature_columns(response_present):
  common_features = ['TMB', 'Systemic_therapy_history', 'Albumin', 'NLR', 'Age'] + \
                    [f'CancerType{i}' for i in range(1, 17)]
  return common_features + ['Response'] if response_present == 'Response' else common_features


Save DataFrame as TSV file.

In [6]:
def save_to_tsv(df, file_path):
  try:
      df.to_csv(file_path, sep='\t', index=False)
  except Exception as e:
      print(f'Error saving {file_path}: {e}')


Read an Excel sheet into a DataFrame with error handling.

In [13]:
def read_sheet(file_path, sheet_name):
  try:
    df = pd.read_excel(excel_file, sheet_name=sheet_name, index_col=0, engine='openpyxl')
    print(f'{sheet_name}: Sheet read completed')
    return df
  except ValueError:
    print(f'Error: Sheet {sheet_name} not found.')

Download the Excel file from a URL and return it as a BytesIO object.

In [8]:
def download_excel(url):
  try:
      response = requests.get(url)
      response.raise_for_status()
      print('Excel file downloaded successfully')
      return BytesIO(response.content)
  except requests.RequestException as e:
      print(f'Error downloading file: {e}')


In [14]:
def list_sheets(excel_file):
    """List available sheet names in the Excel file."""
    try:
        xls = pd.ExcelFile(excel_file, engine='openpyxl')
        print('Available sheets:', xls.sheet_names)
        return xls.sheet_names
    except Exception as e:
        print(f'Error reading sheet names: {e}')
        return None

URL of the Excel file on GitHub

In [16]:
EXCEL_URL = 'https://raw.githubusercontent.com/rootchang/LORIS/main/02.Input/AllData.xlsx'

Download and load the raw dataset from Chang et al. (2024) github repository, then preprocess it to generate files for use with the Galaxy Tool.

After this step, six files (from the Chowell_train, Chowell_test, and MSK1 datasets) will be downloaded and ready to be used as input for Galaxy's machine learning tools.

In [22]:
# Download and load the Excel file
excel_file = download_excel(EXCEL_URL)

if excel_file:  # Proceed if the file was downloaded successfully
    available_sheets = list_sheets(excel_file)  # List sheets in the file

    # Define the sheets you want to load, ensure they exist in the available sheets
    sheets = ['Chowell_train', 'Chowell_test', 'MSK1']
    sheets_to_process = [sheet for sheet in sheets if sheet in available_sheets]

    for sheet in sheets_to_process:
        df = read_sheet(excel_file, sheet)

        if df is not None:  # Check if DataFrame was read successfully
            for feature_option in ['Response', 'No_Response']:
                selected_columns = get_feature_columns(feature_option)
                df_selected = select_and_truncate_columns(selected_columns, df)

                # Save processed data to file
                file_path = os.path.join(DIRECTORY, f'{sheet}_{feature_option}.tsv')
                save_to_tsv(df_selected, file_path)

                # Download the saved TSV file
                print(f'Downloading: {file_path}')
                files.download(file_path)  # Trigger download of the TSV file

Excel file downloaded successfully
Available sheets: ['Chowell_train', 'Chowell_test', 'MSK1', 'MSK2', 'Kato_panCancer', 'Shim_NSCLC', 'Vanguri_NSCLC', 'Ravi_NSCLC', 'Pradat_panCancer', 'MSK_nonICB']
Chowell_train: Sheet read completed
Downloading: /content/Chowell_train_Response.tsv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloading: /content/Chowell_train_No_Response.tsv


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloading: /content/MSK1_No_Response.tsv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>